In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
df_train = pd.read_csv('../input/titanic/train.csv')
df_test = pd.read_csv('../input/titanic/test.csv')

In [3]:
df_train.head()

In [4]:
df_test.head()

In [5]:
df_train.shape

In [6]:
df_test.shape

In [7]:
df_train.info()

In [8]:
df_train.describe()

# Bersiin Data

In [9]:
df_train.isnull().sum()

In [10]:
df_train["Age"].fillna(df_train["Age"].median(skipna=True), inplace=True)

# Data Exploration

In [11]:
# seperate the data into numeric and categorical
df_num = df_train[['Age','SibSp','Parch','Fare']]
df_cat = df_train[['Survived','Pclass','Sex','Ticket','Cabin','Embarked']]

In [12]:
import matplotlib.pyplot as plt
for i in df_num.columns:
    plt.hist(df_num[i])
    plt.title(i)
    plt.show()

In [13]:
import seaborn as sns
sns.heatmap(df_num.corr())

In [14]:
for i in df_cat.columns:
    sns.barplot(df_cat[i].value_counts().index,df_cat[i].value_counts()).set_title(i)
    plt.show()

In [15]:
table_FareAge = pd.pivot_table(df_train, values=['Fare', 'Age'], index=['Sex', 'Survived'],
                       aggfunc={'Fare': [min, max, np.mean],'Age': [min, max, np.mean]})
table_FareAge

In [16]:
table_FareAge

In [17]:
table_FareSex = pd.pivot_table(df_train, columns=[ 'Sex'], index=[ 'Survived', ],
                       aggfunc={ 'Sex': 'count'})
table_FareSex

In [18]:
table_Pclass = pd.pivot_table(df_train, columns = ['Pclass'], index=[ 'Survived']
                      ,  aggfunc={'Pclass': 'count'})
table_Pclass

In [19]:
table_Embarked = pd.pivot_table(df_train, columns = ['Embarked'], index=[ 'Survived']
                      ,  aggfunc={'Embarked': 'count'})
table_Embarked

**Sex (Male --> 0, Female --> 1)**

In [20]:
df_train['Sex'] = df_train.Sex.apply(lambda x: 0 if x == 'male' else 1)

In [21]:
df_train.head()

In [22]:
Selected_features = ['Age', 'Sex', 'Pclass']

# Pembuatan Model

In [23]:
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import accuracy_score, classification_report, precision_score, recall_score 
from sklearn.metrics import confusion_matrix, precision_recall_curve, roc_curve, auc, log_loss
from sklearn.linear_model import LogisticRegression


# create X (features) and y (response)
X = df_train[Selected_features]
y = df_train['Survived']

In [24]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=2)

# check classification scores of logistic regression
logreg = LogisticRegression()
logreg.fit(X_train, y_train)
y_pred = logreg.predict(X_test)
y_pred_proba = logreg.predict_proba(X_test)[:, 1]
[fpr, tpr, thr] = roc_curve(y_test, y_pred_proba)
print('Train/Test split results:')
print(logreg.__class__.__name__+" accuracy is %2.3f" % accuracy_score(y_test, y_pred))
print(logreg.__class__.__name__+" log_loss is %2.3f" % log_loss(y_test, y_pred_proba))
print(logreg.__class__.__name__+" auc is %2.3f" % auc(fpr, tpr))



In [25]:
cf_matrix = confusion_matrix(y_pred, y_test)
import seaborn as sns
sns.heatmap(cf_matrix, annot=True)

In [26]:
input_data = (22,0,1)  # Age, Sex, Pclass [Enter number corelate with this three variable]

In [27]:
input_data_as_numpy_array = np.asarray(input_data)


In [28]:
input_data_reshaped = input_data_as_numpy_array.reshape(1,-1)


In [29]:
prediction = logreg.predict(input_data_reshaped)
#print(prediction)
if prediction[0]==0:
    print("Dead")
if prediction[0]==1:
    print("Alive")